In [43]:
import math
import random

N = 21
d = N

while d > 1:
    x = random.randrange(2, N)
    d = math.gcd(x, N)

N, x

(21, 10)

In [44]:
n = math.ceil(math.log2(N))
t = 2 * n

f"Num qubits: {n + t}"

'Num qubits: 15'

In [45]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFTGate
from quantum.gates import ModularExponentiationGate

qc = QuantumCircuit(t + n, t)
qc.h(range(t))

mod_exponentiation_gate = ModularExponentiationGate(x, N, t, n)
qc.append(mod_exponentiation_gate.get_native(), range(t+n))
qc.append(QFTGate(t), range(t))
qc.measure(range(t), range(t))

In [67]:
from qiskit import transpile
from qiskit_aer import AerSimulator

simulator = AerSimulator()
#qct = transpile(qc, backend=simulator)
counts = simulator.run(qct).result().get_counts()
#qct.count_ops()

In [74]:
from qiskit.visualization import plot_histogram

counts.int_outcomes().pop(0, None)
guess = max(counts.int_outcomes(), key=counts.int_outcomes().get)
{k: v for k, v in sorted(counts.int_outcomes().items(), key=lambda item: item[1], reverse=True)}
#plot_histogram(counts.int_outcomes())

{512: 166,
 171: 122,
 341: 121,
 853: 115,
 683: 114,
 854: 47,
 170: 32,
 682: 27,
 342: 23,
 343: 9,
 340: 8,
 172: 8,
 680: 6,
 852: 6,
 855: 6,
 169: 5,
 685: 4,
 856: 3,
 681: 3,
 684: 3,
 173: 3,
 688: 2,
 339: 2,
 677: 2,
 168: 2,
 174: 2,
 861: 1,
 881: 1,
 167: 1,
 687: 1,
 348: 1,
 162: 1,
 337: 1,
 891: 1,
 679: 1,
 844: 1,
 700: 1,
 689: 1,
 678: 1,
 851: 1,
 850: 1,
 662: 1,
 813: 1,
 875: 1,
 361: 1,
 860: 1,
 334: 1,
 176: 1,
 165: 1,
 839: 1,
 949: 1,
 158: 1,
 841: 1}

In [ ]:
def compute_multiplicative_order(guess, x, N, tol = 1e-8):
    n = math.ceil(math.log2(N))
    q = guess / 2 ** (2 * n)
    print(q)

    # Step k = 0
    a = [math.floor(q), 0]
    q = [q - a[0]]
    b = [a[0], 0, 0]
    c = [1, 0, 0]
    print(a[0], b[0], c[0])
    
    if abs(q[0]) < tol:
        if x ** c[0] % N == 1:
            return 1
        else:
            raise ArithmeticError("Multiplicative order could not be determined from the supplied guess")

    # Step k = 1
    a = [math.floor(1 / q[0]), a[0], 0]
    q = [1 / q[0] - a[0]]
    b = [a[0] * a[1] + 1, b[0], 0]
    c = [a[0], c[0], 0]
    print(a[0], b[0], c[0])

    if x ** c[0] % N == 1:
        return c[0]

    # Step k >= 2
    while abs(q[0]) > tol:
        a = [math.floor(1 / q[0]), a[0], a[1]]
        q = [1 / q[0] - a[0]]
        b = [a[0] * b[0] + b[1], b[0], b[1]]
        c = [a[0] * c[0] + c[1], c[0], c[1]]
        #print(a[0], q[0], b[0], c[0])
        print(a[0], b[0], c[0], x ** c[0] % N)
        
        if x ** c[0] % N == 1:
            return c[0]

    raise ArithmeticError("Multiplicative order could not be determined from the supplied guess")

r = compute_multiplicative_order(171, x, N)

0.1669921875
0 0 1
5 1 5
1 1 6 1


In [84]:
r = 6
assert x ** r % N == 1

assert r % 2 == 0
assert (x ** int(r / 2) + 1) % N != 0

In [85]:
d1 = math.gcd(x ** int(r / 2) + 1, N)
d2 = math.gcd(x ** int(r / 2) - 1, N)

assert math.gcd(d1, N) > 1